# SIT742: Modern Data Science 
**(Assessment Task 01: Wine Rating Data Exploration)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.

Prepared by **SIT742 Teaching Team**


---

**Student Information:** Please fill your information below

- Name: Nnamdi Chukwukelu
- Student ID: 217531243
- Email: nechukwu@deakin.edu.au

---

# Part 0. Data Files

## 0.1 Download Data  

In [1]:
!pip install wget  

In [2]:
import wget

link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Assessment/2019/data/wine.json'
DataSet = wget.download(link_to_data)

link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Assessment/2019/data/stopwords.txt'

DataSet = wget.download(link_to_data)

In [8]:
ls

 Volume in drive C is TI80187200B
 Volume Serial Number is D8E8-A278

 Directory of C:\Users\Nnamdi Chukwukelu\Desktop\6\2019 T1\SIT742- Modern Data Science\sit742-master\Assessment\2019

05/04/2019  09:01 PM    <DIR>          .
05/04/2019  09:01 PM    <DIR>          ..
05/04/2019  08:02 PM    <DIR>          .ipynb_checkpoints
27/03/2019  09:53 PM    <DIR>          data
05/04/2019  09:01 PM            79,954 SIT742Task1.ipynb
05/04/2019  08:02 PM             9,052 SIT742Task2.ipynb
04/04/2019  05:43 PM             4,158 stopwords.txt
04/04/2019  05:43 PM        79,279,294 wine.json
               4 File(s)     79,372,458 bytes
               4 Dir(s)  322,622,410,752 bytes free


## 0.2 Load Data  

In [24]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
file = 'wine.json'

In [5]:
print(file)

wine.json


In [223]:
# write your code here
pd.read_json('wine.json')
wine_data = pd.read_json('wine.json')


# read the json file and drop rows with invalid values in the attributes of 'points' and 'price'.
df = wine_data
import numpy as np

df = df[np.isfinite(df['price'])]
df = df[np.isfinite(df['points'])]
print(type(wine_data))
print(type(df))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


# Part 1: numeric anaysis


## 1.1 Explore the data distribution for each column.

In [85]:
# write your code here
print(df.describe(include='all'), ('dtype'))
# you may use functions such as describe() on each attribute

       country                                        description designation  \
count   120916                                             120975       86196   
unique      42                                             111567       35776   
top         US  This zesty red has pretty aromas that suggest ...     Reserve   
freq     54265                                                  3        1980   
mean       NaN                                                NaN         NaN   
std        NaN                                                NaN         NaN   
min        NaN                                                NaN         NaN   
25%        NaN                                                NaN         NaN   
50%        NaN                                                NaN         NaN   
75%        NaN                                                NaN         NaN   
max        NaN                                                NaN         NaN   

               points      

## 1.2 Find the 10 varieties of wine which receives the highest number of  reviews

In [90]:
# write your code here
# you may use functions such as value_counts()  
top_10= df['variety'].value_counts()
print(top_10[:10])


Pinot Noir                  12787
Chardonnay                  11080
Cabernet Sauvignon           9386
Red Blend                    8476
Bordeaux-style Red Blend     5340
Riesling                     4972
Sauvignon Blanc              4783
Syrah                        4086
Rosé                         3262
Merlot                       3062
Name: variety, dtype: int64


## 1.3 Find varieties of wine having the average price less than 20, with the average pointsat least 90

In [149]:
# write your code here
Avg = df[['variety', 'points', 'price']].groupby('variety').mean().round(decimals = 2)
Avg_pr = Avg.loc[Avg['price'] < 20]
Avg_pr_pt = Avg_pr.loc[Avg_pr['points'] >= 90]
print(Avg_pr_pt)


#groupB = df[['country', 'points', 'price']].groupby('country').mean().round(decimals = 2)
#statTable = pd.concat([groupA, groupB], axis=1).reset_index()
#statTable.columns = ['country', 'variety', 'Av.Points', 'Av.Prices']
# you may use functions such as groupby() 

                          points  price
variety                                
Blauburgunder               93.0  19.00
Caprettone                  92.0  19.00
Kotsifali                   92.0  13.00
Ondenc                      90.0  15.00
Roussanne-Grenache Blanc    91.0  16.00
Shiraz-Malbec               90.0  18.67
Tinta Cao                   90.0  14.00


## 1.4 Build statistic table 

In [124]:
# write your code here
groupA = df[['country', 'variety']].groupby('country').first()
groupB = df[['country', 'points', 'price']].groupby('country').mean().round(decimals = 2)
statTable = pd.concat([groupA, groupB], axis=1).reset_index()
statTable.columns = ['country', 'variety', 'Av.Points', 'Av.Prices']
# you may use functions such as groupby() and round(decimals=2)
print(statTable)

                   country              variety  Av.Points  Av.Prices
0                Argentina               Malbec      86.71      24.51
1                  Armenia              Kangoun      87.50      14.50
2                Australia           Chardonnay      88.60      35.44
3                  Austria     Grüner Veltliner      90.19      30.76
4   Bosnia and Herzegovina               Vranec      86.50      12.50
5                   Brazil      Sparkling Blend      84.66      23.77
6                 Bulgaria                 Rosé      87.94      14.65
7                   Canada                Vidal      89.38      35.71
8                    Chile  Viognier-Chardonnay      86.50      20.79
9                    China       Cabernet Blend      89.00      18.00
10                 Croatia             Malvasia      87.35      25.45
11                  Cyprus          White Blend      87.18      16.27
12          Czech Republic       Welschriesling      87.25      24.25
13                 E

In [ ]:
# save your table to 'statisticByState.csv'

# Part 2. Text Analysis

## 2.1 extract high frequency words in description

In [153]:
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.probability import *
from itertools import chain
from tqdm import tqdm
import codecs

In [154]:
with open('stopwords.txt') as f:
    stop_words = f.read().splitlines()
stop_words = set(stop_words)
#print(stop_words)

In [163]:
# write your code here
description = wine_data['description'] #extracting description from wine_data
description = description.to_string() #conversion of description to string

# define your tokenize

tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?") 
tokens = tokenizer.tokenize(description) #tokenzaition of description strings
tokens = [token.lower() for token in tokens] #mormalizing all to lowercase
#print(tokens[:100]) 
filtered_tokens = [token for token in tokens if token not in stop_words]
#print(filtered_tokens[:100]) #printing first 100 tokens post-removal of stop words

from nltk.stem import PorterStemmer #stemming
stemmer = PorterStemmer()
stemss = ['{1}'.format(w, stemmer.stem(w)) for w in filtered_tokens]
print (stemss[:100]) #priting first 100 stems

['0', 'aroma', 'includ', 'tropic', 'fruit', 'broom', 'brimston', '1', 'ripe', 'fruiti', 'wine', 'smooth', '2', 'tart', 'snappi', 'flavor', 'lime', 'flesh', '3', 'pineappl', 'rind', 'lemon', 'pith', 'orang', 'blossom', '4', 'regular', 'bottl', '2012', '5', 'blackberri', 'raspberri', 'aroma', 'show', 'typic', '6', 'bright', 'inform', 'red', 'open', '7', 'dri', 'restrain', 'wine', 'offer', 'spice', '8', 'savori', 'dri', 'thyme', 'note', 'accent', 'sunnier', 'flavor', '9', 'great', 'depth', 'flavor', 'fresh', '10', 'soft', 'suppl', 'plum', 'envelop', 'oaki', 'structur', '11', 'dri', 'wine', 'spici', 'tight', '12', 'slightli', 'reduc', 'wine', 'offer', 'chalki', '13', 'domin', 'oak', 'oak-driven', 'aroma', '14', 'build', '150', 'year', 'gener', '15', 'zesti', 'orang', 'peel', 'appl', 'note', 'abound', '16', 'bake', 'plum', 'molass', 'balsam', 'vinegar']


In [193]:
# find top common words with document frequencies > 5000
frd = FreqDist(stemss)
top_words = []
for x,y in frd.items():
    if y > 5000:
        print(x,y)
        top_words.append(x)
    
top_words.sort()
print(top_words)
# you may use function FreqDist() and sort()

aroma 20130
fruit 7021
ripe 8027
wine 25798
nose 5025
blend 7428
rich 5099
['aroma', 'blend', 'fruit', 'nose', 'rich', 'ripe', 'wine']


In [194]:
# save your table to 'top_common_words.txt'
with open('top_common_words.txt', 'w') as t:
    for item in top_words:
        t.write("%s\n" % item)

## 2.2 Find key words for describing Shiraz using TF-IDF

In [295]:
#select 'description' from 'variety' equal to  'Shiraz' 
shiraz = wine_data[wine_data.variety == 'Shiraz']
shiraz = dict(shiraz['description'])
print(type(shiraz))

<class 'dict'>


In [298]:
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?") 

shiraz_tokens = {}

for x,y in shiraz.items():
    new_key = x
    new_value = tokenizer.tokenize(str(shiraz.values()))
    shiraz_tokens[new_key] = new_value

In [299]:
# use TfidfVectorizer to calculate TF-IDF score


TypeError: can only join an iterable

In [285]:
# find words with TF-IDF score >0.4 and sort them
vocab =tfidf.get_feature_names()
shiraz_words = []
for word, weight in zip(vocab, tfs.toarray()[0]):
    if weight > 0.4:
        print(word, ":", weight)



NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [ ]:
# save your table to 'key_Shiraz.txt'   